In [3]:
import json
import pandas as pd
import numpy as np
import networkx as nx
import os
import shutil
import subprocess
from scipy.cluster.hierarchy import dendrogram, linkage
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn import preprocessing
from sklearn.cluster import AgglomerativeClustering
from zipfile import ZipFile
from filecmp import dircmp


In [4]:
project_releases = pd.read_csv('complete_links_allVersions_limit20.csv')

In [5]:
project_releases.head()

,project_name,project_link,version_name,commit
0,ZieIony-Carbon,https://github.com/ZieIony/Carbon,0.11.0,"Commit(sha=""7cabd4681b18d88ffa27a848ed76655063..."
1,ZieIony-Carbon,https://github.com/ZieIony/Carbon,0.10.1,"Commit(sha=""68b770b148fe364b701e3d2d4bbfec6e83..."
2,ZieIony-Carbon,https://github.com/ZieIony/Carbon,0.10.0.1,"Commit(sha=""d6a17ed24c096919cf9f55eecd93a04923..."
3,ZieIony-Carbon,https://github.com/ZieIony/Carbon,0.10.0,"Commit(sha=""81b12fa328b86feb517a97bf575528c2f3..."
4,ZieIony-Carbon,https://github.com/ZieIony/Carbon,0.9.0,"Commit(sha=""16ff8ae087e804cd3d5904d55891ace032..."


In [5]:
finished_project_arr = []
finished_project = open('finished_projects.txt','r')
for line in finished_project:
    finished_project_arr.append(line.split('\n')[0])
finished_project.close()
for project_name in project_releases['project_name'].unique():
    max_value_rsf = 0
    if project_name not in finished_project_arr:
        current_project = pd.DataFrame(project_releases[project_releases['project_name'] == project_name])
        current_project['rank'] = current_project['version_name'].rank()
        current_project = current_project.sort_values(by=['rank'], ascending=False)


        ### Download dataset
        try:
            os.mkdir('raw_sourcecode/' + project_name)
        except:
            pass
        for row in current_project.iterrows():
            command = 'cd C:/Users/tanji/Desktop/SoftwareRemodularization/raw_sourcecode/' + project_name +' & mkdir ' + project_name + '_' +row[1]['version_name']

            #print(command)
            os.system(command)

            command = 'git clone ' + row[1]['project_link'] +  ' C:/Users/tanji/Desktop/SoftwareRemodularization/raw_sourcecode/' + project_name + '/' + project_name + '_' + row[1]['version_name']

            #print(command)
            os.system(command)
            commit = row[1]['commit'].replace('Commit(sha="','')
            commit = commit.replace('")','')
            command = 'cd C:/Users/tanji/Desktop/SoftwareRemodularization/raw_sourcecode/' + project_name + '/' + project_name +'_' + row[1]['version_name'] + ' & git checkout ' + commit
            #print(command)
            os.system(command)


        ### Run Depends
        try:
            os.mkdir('raw_depends/' + project_name)
        except:
            pass
        for row in current_project.iterrows():
            #command = 'cd C:/Users/tanji/Desktop/SoftwareRemodularization/raw_depends/' + project_name +' & mkdir ' + project_name + '_' +row[1]['version_name']
            #os.system(command)
            #print(command)
            command = 'cd C:/Users/tanji/Desktop/SoftwareRemodularization/depends-0.9.2 & ' + 'java -jar depends.jar java C:/Users/tanji/Desktop/SoftwareRemodularization/raw_sourcecode/'  + project_name + '/' + project_name + '_' + row[1]['version_name']+  ' ../raw_depends/' + project_name + "/" + project_name + '_' + row[1]['version_name'] 
            #print(command)
            os.system(command)


        try:
            os.mkdir('groundtruth/' + project_name)
        except:
            pass

        try:
            os.mkdir('MoJo_1.2.1/' + project_name)
        except:
            pass


        ### Counter to only ensure the top 10 projects are selected
        counter = 10

        for row in current_project.iterrows():
            counter -= 1
            if counter < 0:
                break
            project_rank = row[1]['rank']
            version_name = row[1]['version_name']
            #print(project_rank)
            rootdir = 'C:/Users/tanji/Desktop/SoftwareRemodularization/raw_sourcecode/' + project_name + '/' + project_name + '_' + row[1]['version_name']

            #print(rootdir)


            ### To obtain the current initial directory
            full_dir_arr = []
            for root, dirs, files in os.walk(rootdir):
                #print(root)
                #print(dirs)
                for element in files:
                    if '.java' in element:
                        dir_string = root + '\\' + element
                        full_dir_arr.append(dir_string)

            cluster_dict = {}
            cluster_tree = {}


            for element in full_dir_arr:
                element = element.split('\\')
                child = element[-1]
                parent = element[-2]
                cluster_tree[child] = parent

            current_rank = project_rank
            try:
                while current_rank >= (project_rank - 9):
                    current_rank -= 1
                    project_to_be_compared = current_project[current_project['rank'] == current_rank]
                    project_to_be_compared_rootdir = 'C:/Users/tanji/Desktop/SoftwareRemodularization/raw_sourcecode/' + project_name + '/' + project_name + '_' + project_to_be_compared['version_name'].values[0]
                    #print(project_to_be_compared_rootdir)


                    ### To obtain the current initial directory
                    project_to_be_compared_full_dir_arr = []
                    for root, dirs, files in os.walk(project_to_be_compared_rootdir):
                        #print(root)
                        #print(dirs)
                        for element in files:
                            if '.java' in element:
                                dir_string = root + '\\' + element
                                project_to_be_compared_full_dir_arr.append(dir_string)

                    project_to_be_compared_cluster_dict = {}
                    project_to_be_compared_cluster_tree = {}


                    for element in project_to_be_compared_full_dir_arr:
                        element = element.split('\\')
                        child = element[-1]
                        parent = element[-2]
                        project_to_be_compared_cluster_tree[child] = parent

                    cluster_tree =  {x:cluster_tree[x] for x in cluster_tree if x in project_to_be_compared_cluster_tree} 

                arr_a_rsf = []
                filename = 'C:/Users/tanji/Desktop/SoftwareRemodularization/groundtruth/' + project_name + '/' + project_name + '_' + row[1]['version_name'] + '.txt'
                with open(filename, 'w') as f:
                    for key, value in cluster_tree.items():
                        arr_a_rsf.append(key)
                        f.write('contain ' + str(value).replace(" ",'') + ' ' + str(key).replace(' ','') + '\n')

                depends_dir = 'C:/Users/tanji/Desktop/SoftwareRemodularization/raw_depends/' + project_name + '/' + project_name + '_' + row[1]['version_name'] + '.json'
                print(depends_dir)

                with open(depends_dir) as f:
                    depends_results = json.load(f)
                    index  = 0
                    var_array = []
                    for value in depends_results['variables']:
                        var_array.append([index, value.split('\\')[-1]])
                        #print(index, value)
                        index += 1

                    var_df = pd.DataFrame(var_array)
                    var_df.columns = ['index_val', 'name']

                    feature_list = {}
                    feature_index = 2
                    for element in depends_results['cells']:
                        #print(element)
                        try:
                            for a in element['values']:
                                if a not in feature_list:
                                    feature_list[a] = feature_index
                                    feature_index += 1
                                #print(a['Call'])


                        except:
                            pass

                    feature_arr = []
                    for element in depends_results['cells']:
                        #print(array)
                        array = [0] * (len(feature_list) + 2)
                        values = dict(element['values'])
                        #print(element)
                        array[0] = element['src']
                        array[1] = element['dest']
                        for feature in feature_list:
                            try:
                                value = values[feature]
                                array[feature_list[feature]] = value
                            except:
                                pass
                        #print(array)
                        feature_arr.append(array)

                    feature_df = pd.DataFrame(feature_arr)
                    col_names = ['src', 'dest']
                    for element in feature_list:
                        col_names.append(element)
                    feature_df.columns = col_names

                    feature_df['sum'] = feature_df.sum(axis=1) - feature_df['src'] - feature_df['dest']
                    G = nx.Graph()
                    for index, row in feature_df.iterrows():
                        G.add_edge(row['src'], row['dest'], weight=row['sum'])

                    adj_mat = nx.adjacency_matrix(G)
                    adj_mat_df = pd.DataFrame(adj_mat.todense())
                    np.fill_diagonal(adj_mat_df.values, adj_mat_df.values.max())
                    x = adj_mat_df.values
                    min_max_scaler = preprocessing.MinMaxScaler()
                    x_scaled = min_max_scaler.fit_transform(x)
                    adj_mat_df = pd.DataFrame(x_scaled)


                    n_cluster_divisible_arr = [5,7,10,15,20,25]
                    affinity_arr = ['euclidean', 'manhattan', 'cosine']
                    linkage_arr = ['complete', 'average', 'single']


                    for user_n_cluster in n_cluster_divisible_arr:
                        for user_affinity in affinity_arr:
                            for user_linkage in linkage_arr:
                                try:
                                    cluster = AgglomerativeClustering(n_clusters=(len(cluster_tree)//user_n_cluster), affinity=user_affinity, linkage=user_linkage)
                                    cluster_result = cluster.fit_predict(adj_mat_df)

                                    filename_a = 'C:/Users/tanji/Desktop/SoftwareRemodularization/MoJo_1.2.1/' + project_name + '/' + project_name + '_' + version_name + '_' + str(user_n_cluster) + '_' +str(user_affinity) + '_' + str(user_linkage) + '_a.rsf'
                                    filename_b = 'C:/Users/tanji/Desktop/SoftwareRemodularization/MoJo_1.2.1/' + project_name + '/' + project_name + '_' + version_name + '_' + str(user_n_cluster) + '_' +str(user_affinity) + '_' + str(user_linkage) + '_b.rsf'

                                    with open(filename_a, 'w') as f:
                                        for key, value in cluster_tree.items():
                                            f.write('contain ' + str(value).replace(" ",'') + ' ' + str(key).replace(' ','') + '\n')

                                    arr_b_rsf = []
                                    with open(filename_b, 'w') as f:
                                        len_b_rsf = 0
                                        duplicate_array = []
                                        for i in range(len(cluster_result)):
                                            try:
                                                subject = var_df[var_df['index_val'] == i]['name'].values[0]
                                                #print(subject)
                                                #print(subject in cluster_tree)
                                                if subject in cluster_tree and subject not in duplicate_array:
                                                    #print(element)
                                                    duplicate_array.append(subject)
                                                    string = "contain " + str(cluster_result[i]) + " " + subject + "\n"
                                                    arr_b_rsf.append(subject)
                                                    len_b_rsf += 1
                                                    f.write(string)
                                                else:
                                                    #print(subject)
                                                    pass
                                            except:
                                                pass
                                    f.close()

                                    len_a_rsf = len(cluster_tree)

                                    if len_b_rsf > len_a_rsf:
                                        rsf_initial_counter = len_a_rsf
                                        to_be_added = list(set(arr_a_rsf) - set(arr_b_rsf))
                                        with open(filename_a,'a+') as f:

                                            for item in to_be_added:
                                                string = 'contain ' + str(rsf_initial_counter) + " " + item + '\n'
                                                rsf_initial_counter += 1
                                                f.write(string)
                                        f.close()

                                    elif len_a_rsf > len_b_rsf:

                                        rsf_initial_counter = len_b_rsf
                                        to_be_added = list(set(arr_a_rsf) - set(arr_b_rsf))
                                        with open(filename_b,'a+') as f:

                                            for item in to_be_added:
                                                string = 'contain ' + str(rsf_initial_counter) + " " + item + '\n'
                                                rsf_initial_counter += 1
                                                f.write(string)
                                        f.close()

                                    command = 'cd C:/Users/tanji/Desktop/SoftwareRemodularization/MoJo_1.2.1 & ' + 'java MoJo ' + filename_a + ' ' + filename_b + ' >> ' + project_name + '/' + project_name + '_results.txt'
                                    #print(command)
                                    os.system(command)

                                    if max_value_rsf < len_a_rsf:
                                        max_value_rsf = len_a_rsf
                                    if max_value_rsf < len_b_rsf:
                                        max_value_rsf = len_b_rsf

                                except:
                                    pass



                f.close()
            except:
                pass
        finished_project = open('finished_projects.txt','a+')
        finished_project.write(project_name + '\n')
        finished_project.close()
        
        max_len_txt = open('C:/Users/tanji/Desktop/SoftwareRemodularization/MoJo_1.2.1/' + project_name + '/' + project_name + '_max_len.txt', 'w')
        max_len_txt.write(str(max_value_rsf) + '\n')
        max_len_txt.close()
        try:
            shutil.copy('MoJo_1.2.1/' + project_name + '/' + project_name + '_results.txt', 'Results')
            shutil.copy('MoJo_1.2.1/' + project_name + '/' + project_name +  '_max_len.txt', 'Results')
        except:
            pass

git clone https://github.com/ZieIony/Carbon C:/Users/user/Desktop/SoftwareRemodularization/raw_sourcecode/ZieIony-Carbon/ZieIony-Carbon_7.1.84
cd C:/Users/user/Desktop/SoftwareRemodularization/raw_sourcecode/ZieIony-Carbon/ZieIony-Carbon_7.1.84 & git checkout 567fd84
git clone https://github.com/ZieIony/Carbon C:/Users/user/Desktop/SoftwareRemodularization/raw_sourcecode/ZieIony-Carbon/ZieIony-Carbon_7.1.83
cd C:/Users/user/Desktop/SoftwareRemodularization/raw_sourcecode/ZieIony-Carbon/ZieIony-Carbon_7.1.83 & git checkout 9a343f5
git clone https://github.com/ZieIony/Carbon C:/Users/user/Desktop/SoftwareRemodularization/raw_sourcecode/ZieIony-Carbon/ZieIony-Carbon_7.1.82
cd C:/Users/user/Desktop/SoftwareRemodularization/raw_sourcecode/ZieIony-Carbon/ZieIony-Carbon_7.1.82 & git checkout a655476
git clone https://github.com/ZieIony/Carbon C:/Users/user/Desktop/SoftwareRemodularization/raw_sourcecode/ZieIony-Carbon/ZieIony-Carbon_7.1.81
cd C:/Users/user/Desktop/SoftwareRemodularization/raw

KeyboardInterrupt: 

In [7]:
for row in project_releases.iterrows():
    commit = row[1]['commit'].replace('Commit(sha="','')
    commit = commit.replace('")','')
    print(commit)